
# Space

In [1]:
import os
import sys 
import logging
import pandas as pd 
from pprint import pprint 

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)

# Available Packages
import pandas as pd
from datetime import datetime 
import logging
logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')

g:\Shared drives\CDHAI-WellDoc\2024-WellDocTest-SPACE\_WellDoc-AI-CGMGPT-WorkSpace
{'CODE_FN': '../pipeline',
 'CODE_RFT': '../pipeline',
 'DATA_CaseFeat': '../_Data/3-Data_CaseFeat',
 'DATA_CaseObs': '../_Data/2-Data_CaseObs',
 'DATA_CaseSet': '../_Data/4-Data_CaseSet',
 'DATA_RAW': '../_Data/0-Data_Raw',
 'DATA_RFT': '../_Data/1-Data_RFT',
 'DATA_TASK': '../_Data/8-Data_AIDataset',
 'MODEL_TASK': '../_Model',
 'WORKSPACE_PATH': 'g:\\Shared '
                   'drives\\CDHAI-WellDoc\\2024-WellDocTest-SPACE\\_WellDoc-AI-CGMGPT-WorkSpace'}


# Trigger Events

In [2]:
import datasets
from recfldtkn.loadtools import load_ds_rec_and_info
from recfldtkn.configfn import load_cohort_args, load_record_args
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

base_config = load_cohort_args(recfldtkn_config_path, SPACE)
print(base_config)

{'CohortInfo': {'RawData2022_CGM': {'cohort_label': 1, 'cohort_name': 'RawData2022_CGM', 'FolderPath': '../_Data/0-Data_Raw/RawData2022_CGM/'}, 'RawData2023_CVSTDCAug': {'cohort_label': 2, 'cohort_name': 'RawData2023_CVSTDCAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/'}, 'RawData2023_CVSDeRxAug': {'cohort_label': 3, 'cohort_name': 'RawData2023_CVSDeRxAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSDeRxAug/'}}, 'RawRootID': 'PatientID', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_to_RFT_GROUP_SIZE': {'CGM5Min': 100, 'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'CGM5Min': 100, 'BGManual': 100, 'Default': 10000}, 'RecName_to_RFT_usebucket': {'CGM5Min': True, 'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_p

In [3]:
# CaseFolder = os.path.join(SPACE['DATA_TASK'], 'CaseFolder')
# print(CaseFolder)

# Trigger Case

In [4]:
#################
cohort_label_list = [1, 2, 3]
#################

######################################
TriggerCaseMethod = 'FoodEntryEOD'
######################################

TriggerRecName = 'FoodRec'
case_id_columns = ["PID", "ObsDT"]
special_columns = ['PID', 'DT_s']

In [5]:

import inspect

def convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config):
    import logging
    logger = logging.getLogger(__name__)
    from datetime import timedelta
    ds_case = ds_rec.select_columns(special_columns)
    df_case_raw = ds_case.to_pandas()
    
    # ------------------------------ Filter Cases ------------------------------
    df_case_filter = df_case_raw # [df_case_raw['invitation_state'] == 'Delivered'].reset_index(drop=True)
    # --------------------------------------------------------------------------

    # ------------------------------ Determine the ObsDT ------------------------------
    df_case_filter['ObsDT'] = df_case_filter['DT_s'] # [df_case_raw['drug_name'] == 'Trulicity']
    df_case_filter['ObsDT'] = df_case_filter['ObsDT'].dt.normalize() + timedelta(days=1)
    df_case_filter = df_case_filter.drop(columns = 'DT_s') # .from_pandas(df_case_filter)
    
    
    
    # ------------------------------- Update Column Sequence ------------------------
    columns = df_case_filter.columns 
    columns = case_id_columns + [col for col in columns if col not in case_id_columns]
    df_case_filter = df_case_filter[columns].reset_index(drop=True)

    # --------------------------------------------------------------------------
    logger.info('Number of Cases before dropping: {}'.format(len(df_case_filter)))
    df_case_filter = df_case_filter.drop_duplicates().reset_index(drop = True)
    logger.info('Number of Cases after  dropping: {}'.format(len(df_case_filter)))
    
    df_case = df_case_filter
    return df_case

convert_TriggerEvent_to_Caseset.fn_string = inspect.getsource(convert_TriggerEvent_to_Caseset)

In [6]:
prefix = ['import pandas as pd', 'import numpy as np']
string_variables = [TriggerRecName]
iterative_variables = [case_id_columns, special_columns]
fn_variables = [convert_TriggerEvent_to_Caseset]
pycode = convert_variables_to_pystirng(string_variables = string_variables, 
                                       iterative_variables = iterative_variables, 
                                       fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(base_config['trigger_pyfolder'], f'{TriggerCaseMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

../pipeline/fn_trigger/FoodEntryEOD.py


In [7]:
pypath = os.path.join(base_config['trigger_pyfolder'], f'{TriggerCaseMethod}.py')
module = load_module_variables(pypath)

######################
TriggerRecName = module.TriggerRecName
case_id_columns = module.case_id_columns
special_columns = module.special_columns
convert_TriggerEvent_to_Caseset = module.convert_TriggerEvent_to_Caseset
######################

In [8]:
############################### ds_rec to df_case ###############################
ds_rec, _ = load_ds_rec_and_info(TriggerRecName, base_config, cohort_label_list)
ds_rec 

Dataset({
    features: ['PID', 'PatientID', 'FoodRecID', 'DT_s', 'DT_r', 'DT_tz', 'EntryType', 'FoodName', 'ServingSize', 'ServingType', 'ServingsConsumed', 'Carbs', 'Fiber', 'Fat', 'Calories', 'Protein', 'Sodium', 'SaturatedFat', 'PolyUnSaturatedFat', 'MonoUnSaturatedFat', 'TransFat', 'Cholesterol', 'Potassium', 'Sugar', 'AddedSugars', 'ActivityTypeID', 'ExternalSourceID', 'ObservationStatus', 'FoodRec-SysCateTkn_wgt', 'FoodRec-SysCateTkn_tknidx', 'FoodRec-NutriN2CTkn_wgt', 'FoodRec-NutriN2CTkn_tknidx', 'FoodRec-TypeCateTkn_wgt', 'FoodRec-TypeCateTkn_tknidx', 'FoodRec-NutriNumeTkn_wgt', 'FoodRec-NutriNumeTkn_tknidx'],
    num_rows: 142022
})

In [9]:
ds_rec.to_pandas()

,PID,PatientID,FoodRecID,DT_s,DT_r,DT_tz,EntryType,FoodName,ServingSize,ServingType,...,ExternalSourceID,ObservationStatus,FoodRec-SysCateTkn_wgt,FoodRec-SysCateTkn_tknidx,FoodRec-NutriN2CTkn_wgt,FoodRec-NutriN2CTkn_tknidx,FoodRec-TypeCateTkn_wgt,FoodRec-TypeCateTkn_tknidx,FoodRec-NutriNumeTkn_wgt,FoodRec-NutriNumeTkn_tknidx
0,1000001,6,1000001-0,2021-02-15 22:58:00,2021-02-16 22:58:48,-300,Manu,Apple Sauce,0.5,cup,...,33,1,"[1, 1, 1, 1]","[6, 10, 13, 19]","[1.0, 0.5, 1.0, 1.0, 0.8, 1.0, 1.0, 1.0, 0.7, ...","[21, 22, 49, 119, 120, 9, 7, 1, 2, 17, 25, 19,...","[1, 1]","[2, 36]","[2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[1, 2, 3, 4, 5, 22, 7, 8, 9, 10, 11, 12, 13, 1..."
1,1000001,6,1000001-1,2021-02-16 22:56:00,2021-02-16 22:56:56,-300,Manu,Apple Sauce,0.5,cup,...,33,1,"[1, 1, 1, 1]","[6, 10, 13, 19]","[1.0, 0.5, 1.0, 1.0, 0.8, 1.0, 1.0, 1.0, 0.7, ...","[21, 22, 49, 119, 120, 9, 7, 1, 2, 17, 25, 19,...","[1, 1]","[2, 36]","[2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[1, 2, 3, 4, 5, 22, 7, 8, 9, 10, 11, 12, 13, 1..."
2,1000001,6,1000001-2,2021-02-17 00:19:00,2021-02-17 00:19:53,-300,Manu,Crispy Colonel's Sandwich - Honey BBQ,1.0,Sandwich,...,26,1,"[1, 1, 1, 1]","[5, 10, 16, 19]","[1.0, 1.0, 1.0, 0.8, 1.0, 1.0, 1.0, 0.1, 1.0, ...","[35, 37, 222, 223, 31, 190, 357, 358, 172, 173...","[1, 1]","[2, 34]","[1.0, 0.0, 25.0, 12.0, 0.0, 1.0, 60.0, 1.0, 12...","[1, 2, 3, 4, 5, 22, 7, 8, 9, 10, 11, 12, 13, 1..."
3,1000001,6,1000001-3,2021-02-17 00:30:00,2021-02-17 00:30:26,-300,Manu,Lipton Brisk Sweet Iced Tea (30 fl oz),30.0,Fl. Oz.,...,26,1,"[1, 1, 1, 1]","[5, 10, 15, 19]","[1.0, 1.0, 1.0, 0.4, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[509, 79, 236, 237, 9, 7, 312, 17, 290, 291, 1...","[1, 1]","[2, 34]","[5.0, 0.0, 0.0, 54.0, 0.0, 1.0, 0.0, 0.0, 120....","[1, 2, 3, 4, 5, 22, 7, 8, 9, 10, 11, 12, 13, 1..."
4,1000001,6,1000001-4,2021-02-17 09:20:00,2021-02-17 10:47:58,330,Manu,Beef Burger,1.0,burger,...,33,2,"[1, 1, 1, 1]","[6, 10, 13, 20]","[1.0, 1.0, 1.0, 0.38, 1.0, 1.0, 0.3, 1.0, 0.84...","[35, 49, 222, 223, 9, 210, 211, 357, 358, 213,...","[1, 1]","[2, 34]","[2.0, 0.0, 31.51, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0...","[1, 2, 3, 4, 5, 22, 7, 8, 9, 10, 11, 12, 13, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142017,3000069,4113,3000069-319,2023-04-12 07:49:00,2023-04-12 09:49:44,-420,Manu,Turkey Sandwich,1.0,sandwich,...,14,2,"[1, 1, 1, 1]","[3, 10, 13, 20]","[1.0, 1.0, 0.5, 1.0, 0.67, 1.0, 0.2, 1.0, 0.79...","[35, 23, 24, 164, 165, 31, 32, 71, 72, 330, 33...","[1, 1]","[2, 42]","[0.5, 309.0, 8.94, 2.41, 0.01, -1.0, 64.0, 1.2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
142018,3000069,4113,3000069-320,2023-04-12 14:55:00,2023-04-12 20:55:51,-420,Manu,Breakfast Burrito,1.0,burrito,...,14,1,"[1, 1, 1, 1]","[3, 10, 14, 19]","[1.0, 1.0, 1.0, 0.8, 1.0, 0.2, 1.0, 0.71, 1.0,...","[35, 37, 119, 120, 31, 32, 146, 147, 330, 331,...","[1, 1]","[2, 34]","[1.0, 234.0, 18.57, 2.63, 0.0, -1.0, 267.0, 1....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
142019,3000069,4113,3000069-321,2023-04-12 20:56:00,2023-04-12 20:56:22,-420,Manu,Taco Salad,1.5,cup,...,14,1,"[1, 1, 1, 1]","[3, 10, 15, 19]","[1.0, 0.5, 1.0, 1.0, 0.36, 1.0, 1.0, 0.95, 1.0...","[35, 36, 49, 164, 165, 9, 121, 122, 312, 313, ...","[1, 1]","[2, 36]","[2.0, 416.0, 14.77, 0.0, 0.0, -1.0, 44.0, 0.0,...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
142020,3000069,4113,3000069-322,2023-04-13 07:37:00,2023-04-13 07:38:03,-420,Manu,Turkey Sandwich,1.0,sandwich,...,14,1,"[1, 1, 1, 1]","[3, 10, 13, 19]","[1.0, 1.0, 0.5, 1.0, 0.67, 1.0, 0.2, 1.0, 0.79...","[35, 23, 24, 164, 165, 31, 32, 71, 72, 330, 33...","[1, 1]","[2, 42]","[0.5, 309.0, 8.94, 2.41, 0.01, -1.0, 64.0, 1.2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [10]:
df_case = convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config)
df_case

[INFO:2024-04-21 09:05:31,006:(FoodEntryEOD.py@35 FoodEntryEOD)]: Number of Cases before dropping: 142022
[INFO:2024-04-21 09:05:31,013:(FoodEntryEOD.py@37 FoodEntryEOD)]: Number of Cases after  dropping: 20917


,PID,ObsDT
0,1000001,2021-02-16
1,1000001,2021-02-17
2,1000001,2021-02-18
3,1000001,2021-02-22
4,1000001,2021-02-23
...,...,...
20912,3000069,2023-04-08
20913,3000069,2023-04-11
20914,3000069,2023-04-12
20915,3000069,2023-04-13


In [11]:
df_case['PID'].sort_values()

0        1000001
1        1000001
2        1000001
3        1000001
4        1000001
          ...   
20866    3000069
20865    3000069
20864    3000069
20880    3000069
20916    3000069
Name: PID, Length: 20917, dtype: int64

In [12]:
 #  = [1, 2, 3]

In [13]:
CaseFolder = os.path.join(SPACE['DATA_CaseSet'])

if not os.path.exists(CaseFolder): os.makedirs(CaseFolder)

cohort_label_list_str = '_'.join([str(i) for i in cohort_label_list])
# path = os.path.join(CaseFolder, f'{TriggerCaseMethod}-{cohort_label_list_str}.p')
path = os.path.join(CaseFolder, f'{TriggerCaseMethod}.p')
df_case.to_pickle(path)
print(path)

../_Data/4-Data_CaseSet\FoodEntryEOD.p
